## HW5: SQLAlchemy + MediaCloud 

In [1]:
#Initialize sqlalchemy

from sqlalchemy import *
import datetime

In [2]:
## Initialize mediacloud
import mediacloud, json, datetime
from config import MY_API_KEY

mc = mediacloud.api.MediaCloud(MY_API_KEY)

keyword1 = 'trump'
res = mc.sentenceCount(keyword1, solr_filter=[mc.publish_date_query( datetime.date( 2017, 11, 1), datetime.date( 2017, 11, 30) ), 'tags_id_media:1' ])
query1 = {'keyword': keyword1, 'count': res['count']}
print(query1)

keyword2 = 'clinton'
res = mc.sentenceCount(keyword2, solr_filter=[mc.publish_date_query( datetime.date( 2017, 11, 1), datetime.date( 2017, 11, 30) ), 'tags_id_media:1' ])
query2 = {'keyword': keyword2, 'count': res['count']}
print(query2)


{'keyword': 'trump', 'count': 174757}
{'keyword': 'clinton', 'count': 17963}


In [4]:

#Create a table 

# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:',echo=True) # just save the db in memory for now (ie. not on disk)
db_conn = engine.connect()
metadata = MetaData()
# define a table to use
keywords_table = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keyword', String(400), nullable=False),
    Column('count', Integer, nullable=False),
    Column('timestamp', DateTime, default=datetime.datetime.now),
)
metadata.create_all(engine) # and create the tables in the databa


2017-12-14 01:24:53,038 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-12-14 01:24:53,040 INFO sqlalchemy.engine.base.Engine ()
2017-12-14 01:24:53,044 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-12-14 01:24:53,047 INFO sqlalchemy.engine.base.Engine ()
2017-12-14 01:24:53,052 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("queries")
2017-12-14 01:24:53,054 INFO sqlalchemy.engine.base.Engine ()
2017-12-14 01:24:53,059 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE queries (
	id INTEGER NOT NULL, 
	keyword VARCHAR(400) NOT NULL, 
	count INTEGER NOT NULL, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2017-12-14 01:24:53,063 INFO sqlalchemy.engine.base.Engine ()
2017-12-14 01:24:53,065 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
# Insert count per keyword
insert_stmt = keywords_table.insert().values([query1, query2])
result = db_conn.execute(insert_stmt)

2017-12-14 01:24:57,553 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keyword, count, timestamp) VALUES (?, ?, ?), (?, ?, ?)
2017-12-14 01:24:57,555 INFO sqlalchemy.engine.base.Engine ('trump', 174757, '2017-12-14 01:24:57.553210', 'clinton', 17963, '2017-12-14 01:24:57.553219')
2017-12-14 01:24:57,563 INFO sqlalchemy.engine.base.Engine COMMIT


In [6]:
# Select count per keyword
# Show total

from sqlalchemy.sql import select

select_stmt = select([keywords_table])
results = db_conn.execute(select_stmt)

total_count = 0

for row in results:
    total_count += row.count
    print('The count for {} is {}'.format(row.keyword, row.count))

print('Total count is {}'.format(total_count))
    

2017-12-14 01:25:16,998 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keyword, queries.count, queries.timestamp 
FROM queries
2017-12-14 01:25:17,000 INFO sqlalchemy.engine.base.Engine ()
The count for trump is 174757
The count for clinton is 17963
Total count is 192720
